In [1]:
%LOAD test.db

## High Level Sales Analysis

### 1. What was the total quantity sold for all products?

In [9]:
SELECT SUM(qty) AS total_qty
FROM sales;

total_qty
45216


### 2. What is the total generated revenue for all products before discounts?

In [10]:
SELECT SUM(qty*price) AS total_revenue
FROM sales;

total_revenue
1289453


### 3. What was the total discount amount for all products?

In [11]:
SELECT SUM((qty*price)*(1-discount/100)) AS total_discount
FROM sales;

total_discount
1289453


## Transaction Analysis

### 1. How many unique transactions were there?

In [12]:
SELECT COUNT (DISTINCT(txn_id)) AS unique_transactions
FROM sales;

unique_transactions
2500


### 2. What is the average unique products purchased in each transaction?

In [15]:
SELECT AVG(counts) AS avg_unique_products
FROM (
  SELECT COUNT (DISTINCT(prod_id)) AS counts
  FROM sales
  GROUP BY txn_id) AS subquery;

avg_unique_products
6.038


### 3. What are the 25th, 50th and 75th percentile values for the revenue per transaction?
This works in SQL, but the kernel used here is SQLite, which does not recognise PERCENTILE_CONT.

In [19]:
SELECT
PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY price) AS percentile25,
PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY price) AS percentile50,
PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY price) AS percentile75
FROM sales
GROUP BY txn_id;

Error: near "(": syntax error

### 4. What is the average discount value per transaction?

In [63]:
SELECT txn_id, AVG(discount)
FROM sales
GROUP BY txn_id
LIMIT 5;

txn_id,AVG(discount)
000027,13.0
000106,12.0
000dd8,6.0
003920,18.0
003c6d,13.0


### 5. What is the percentage split of all transactions for members vs non-members?

In [128]:
SELECT member, ROUND(COUNT(member) * 100.0/
    (SELECT COUNT(member) 
     FROM SALES), 2) AS percentage
FROM sales
GROUP BY member;

member,percentage
f,39.97
t,60.03


### 6. What is the average revenue for member transactions and non-member transactions?

In [62]:
SELECT member, AVG(qty*price) AS avg_revenue
FROM sales
GROUP BY member;

member,avg_revenue
f,84.9302287040106
t,85.7503586800574


## Product Analysis

### 1. What are the top 3 products by total revenue before discount?

In [30]:
SELECT pd.product_name
FROM sales as s
INNER JOIN 
product_details as pd
ON s.prod_id = pd.product_id
GROUP BY s.prod_id
ORDER BY SUM(s.qty*s.price) DESC
LIMIT 3;

product_name
Blue Polo Shirt - Mens
Grey Fashion Jacket - Womens
White Tee Shirt - Mens


### 2. What is the total quantity, revenue and discount for each segment?

In [26]:
SELECT pd.segment_name, SUM(s.qty) AS qty, SUM((s.qty*s.price)*(1-s.discount/100)) AS revenue, 
        SUM(s.discount) AS discount
FROM sales AS s
INNER JOIN
product_details as pd
ON s.prod_id = pd.product_id
GROUP BY pd.segment_name;

segment_name,qty,revenue,discount
Jacket,11385,366983,45452
Jeans,11349,208350,45740
Shirt,11265,406143,46043
Socks,11217,307977,45465


### 3. What is the top selling product for each segment?

In [33]:
SELECT pd.segment_name, pd.product_name
FROM sales as s
INNER JOIN 
product_details as pd
ON s.prod_id = pd.product_id
GROUP BY pd.segment_name
ORDER BY s.qty DESC;

segment_name,product_name
Socks,White Striped Socks - Mens
Shirt,White Tee Shirt - Mens
Jeans,Navy Oversized Jeans - Womens
Jacket,Indigo Rain Jacket - Womens


### 4. What is the total quantity, revenue and discount for each category?

In [31]:
SELECT pd.category_name, SUM(s.qty) AS qty, SUM((s.qty*s.price)*(1-s.discount/100)) AS revenue, 
        SUM(s.discount) AS discount
FROM sales AS s
INNER JOIN
product_details as pd
ON s.prod_id = pd.product_id
GROUP BY pd.category_name;

category_name,qty,revenue,discount
Mens,22482,714120,91508
Womens,22734,575333,91192


### 5. What is the top selling product for each category?

In [34]:
SELECT pd.category_name, pd.product_name
FROM sales as s
INNER JOIN 
product_details as pd
ON s.prod_id = pd.product_id
GROUP BY pd.category_name
ORDER BY s.qty DESC;

category_name,product_name
Womens,Navy Oversized Jeans - Womens
Mens,White Tee Shirt - Mens


### 6. What is the percentage split of revenue by product for each segment?

In [120]:
SELECT pd.product_name, ROUND(SUM(s.qty*s.price*1.0)/
    (SELECT SUM(s2.qty*s2.price*1.0) 
     FROM sales AS s2
     INNER JOIN product_details as pd2
     ON s2.prod_id = pd2.product_id 
     WHERE pd.segment_name = pd2.segment_name), 2) AS percentage
FROM sales AS s
INNER JOIN product_details as pd
ON s.prod_id = pd.product_id 
GROUP BY pd.product_name
ORDER BY pd.segment_name;

product_name,percentage
Grey Fashion Jacket - Womens,0.57
Indigo Rain Jacket - Womens,0.19
Khaki Suit Jacket - Womens,0.24
Black Straight Jeans - Womens,0.58
Cream Relaxed Jeans - Womens,0.18
Navy Oversized Jeans - Womens,0.24
Blue Polo Shirt - Mens,0.54
Teal Button Up Shirt - Mens,0.09
White Tee Shirt - Mens,0.37
Navy Solid Socks - Mens,0.44


### 7. What is the percentage split of revenue by segment for each category?

In [121]:
SELECT pd.segment_name, ROUND(SUM(s.qty*s.price*1.0)/
    (SELECT SUM(s2.qty*s2.price*1.0) 
     FROM sales AS s2
     INNER JOIN product_details as pd2
     ON s2.prod_id = pd2.product_id 
     WHERE pd.category_name = pd2.category_name), 2) AS percentage
FROM sales AS s
INNER JOIN product_details as pd
ON s.prod_id = pd.product_id 
GROUP BY pd.segment_name
ORDER BY pd.category_name;

segment_name,percentage
Shirt,0.57
Socks,0.43
Jacket,0.64
Jeans,0.36


### 8. What is the percentage split of total revenue by category?

In [124]:
SELECT ROUND(SUM(s.qty*s.price*1.0)/
             (SELECT SUM(qty*price*1.0)
              FROM sales), 2) AS percentage
FROM sales AS s
INNER JOIN product_details as pd
ON s.prod_id = pd.product_id 
GROUP BY pd.category_name

percentage
0.55
0.45


### 9. What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)

In [147]:
SELECT pd.product_name, ROUND(COUNT(txn_id) * 1.0/
    (SELECT COUNT(txn_id)*1.0 
     FROM sales), 6) AS percentage
FROM sales AS s
INNER JOIN product_details as pd
ON s.prod_id = pd.product_id 
GROUP BY prod_id
ORDER BY percentage DESC

product_name,percentage
Navy Solid Socks - Mens,0.084863
Grey Fashion Jacket - Womens,0.084465
Navy Oversized Jeans - Womens,0.084399
White Tee Shirt - Mens,0.084001
Blue Polo Shirt - Mens,0.084001
Pink Fluro Polkadot Socks - Mens,0.083339
Indigo Rain Jacket - Womens,0.082809
Khaki Suit Jacket - Womens,0.08261
Black Straight Jeans - Womens,0.082544
Cream Relaxed Jeans - Womens,0.082345


### 10. What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

In [23]:
WITH combination AS (
    SELECT DISTINCT prod_id, product_name
    FROM sales AS s
    INNER JOIN product_details AS pd
    ON pd.product_id = s.prod_id
    ORDER BY prod_id)
    
SELECT c1.product_name, c2.product_name, c3.product_name, 
    (SELECT COUNT(*)
    FROM sales as s1
    INNER JOIN sales as s2
    ON s1.txn_id = s2.txn_id
    INNER JOIN sales as s3
    ON s1.txn_id = s3.txn_id
    WHERE s1.prod_id = c1.prod_id AND
    s2.prod_id = c2.prod_id AND s3.prod_id = c3.prod_id) AS count 
FROM combination AS c1, combination as c2, combination as c3
WHERE c1.prod_id < c2.prod_id AND
    c2.prod_id < c3.prod_id 
ORDER BY count DESC
LIMIT 1;

product_name,product_name,product_name,count
White Tee Shirt - Mens,Grey Fashion Jacket - Womens,Teal Button Up Shirt - Mens,352
